In [7]:
import numpy as np 
import pandas as pd 
import os 
import h5py 
import csv 
import datetime as dt

In [8]:
file_loc = 'iel.hdf5'                   #Location of the hdf5 file

<h2><font color =#bf5700>UT Austin Dataset</font></h2>

<h3><font color = #333f48>Data Transformation</font></h3>

The structure of the data files is segregated by building and then again by year. For example, 2015 data for the Welch building is located in 'utexas/WEL/2015.csv'

In [9]:
home = 'utexas'                         #Insert home directory where data is stored
ext = '.csv'                            #Insert data storage type
separ = ','                             #Separation
mdata = 'utexas/utexas_metadata.csv'    #Point to location of metadata file

In [10]:
root, folders,x = [m for m in os.walk(home)][0]
folder_paths = [os.path.join(home, subdir) for subdir in folders]
subfiles = []
for f in folder_paths: 
    r,a, data = [m for m in os.walk(f)][0]
    for d in data: 
        if os.path.splitext(d)[1]==ext:
            subfiles.append(os.path.join(r,d))
hdf_dsets = [s.replace('\\','/').replace(ext,'') for s in subfiles]

'Subfiles' should now have a list of the complete path of all data files
Ut dataset involves various time formats. convert_any_time converts string formats to pd datetime while float formats (already in pd format) are left alone: 

In [11]:
def convert_any_dtime(x):
    try:
        return dt.datetime.strptime(x, "%m/%d/%y %H:%M")
    except:
        try:
            return dt.datetime.strptime(x, "%m/%d/%Y %H:%M")
        except:
            return None
def read_mdata(mdata_file):
    return pd.read_csv(mdata_file,index_col=0)

In [12]:
meta_db = read_mdata(mdata)

Below is the code that compiles the data into the file. Be warned, data processing may take over 20 minutes. For a sense of where in the process the computer is, remove the commented portions

In [ ]:
with h5py.File(file_loc,'a') as f:
    #counter = 0
    for data_file in subfiles: 
        #counter = counter+1
        #print('Now reading dataset #{}......................{}'.format(counter,data_file))
        d = pd.read_csv(data_file).dropna(axis=0,how='any',inplace=False)
        dt_dates = [convert_any_dtime(x) if type(x)==str else x for x in d['DateTime'] ]
        dates= np.array( [x.strftime("%Y-%m-%d %H:%M:%S").encode('utf8') for x in dt_dates], dtype = np.string_).reshape((-1,1))
        usage =np.array([x for x in d['Electrical ( kWh )']],dtype=np.float64).reshape((-1,1))
        hdfs_dset = data_file.replace('\\','/').replace(ext,'')
        f[hdfs_dset] = np.hstack((dates,usage))  

<h3><font color = #333f48>Metadata Transformation</font></h3>

In [ ]:
with h5py.File(file_loc,'a') as f:
    for building in meta_db.index:
        psu,sqft = meta_db.loc[building]
        if '{}/{}'.format(home,building) in f:
            bldg_grp = f['{}/{}'.format(home,building)]
            bldg_grp.attrs['PSU'] = psu
            bldg_grp.attrs['Sqft'] = sqft

<h2><font color =#bf5700>MIT Dataset</font></h2>

In [14]:
home = 'MIT'                            #Insert home directory where data is stored
ext = '.csv'                            #Insert data storage type
separ = ','                             #Separation
mdata = '#'                             #Point to location of metadata file

<h3><font color = #333f48>Data Transformation</font></h3>

In [15]:
root, folders,files = [m for m in os.walk(home)][0]
subfiles = []
for d in files: 
    if os.path.splitext(d)[1]==ext:
        subfiles.append(os.path.join(root,d))

print(files)

['2014.csv', '2015.csv', '2016.csv']


The MIT dataset is shaped differently than our desires in a few ways: 
<ul>
<li>The files are segregated by year instead of by building</li>
<li>The files contain multiple 'error' values, so we need to screen more than just N/A</li>
<li>The list is NOT ordered, meaning that time intervals next to each other in the original scrape will NOT necessarily be together in the final </li>
<li>In addition the separation between timestamps is not hourly as we would like but every 15 minutes</li>
<li> To further complicate matters, the list increments are in kilowatts (kW), a unit of power, not a unit of energy!!!  Power is related to energy usage in that power is average energy usage over a period of time. A Watt is 1 Joule per Second, and so 1 <b>kilowatt hour</b> is our standard measurement of energy: as
$$1\text{ kWH} =\frac{\text{1 Joule}}{\text{1 second}}\times \text{3600 seconds} = 3600\text{ Joule}$$
A 1 kW power usage for 15 minutes is therefore $\frac{1}{4} \times 3600 \text{ Joule} = 900 \text{Joule}$
Thus to find the total power usage over 4 different 15 minute intervals, take the average of the power usage of each individual file 
</ul>

In [16]:
def mit_name_clean(colname):
    colname = colname.replace('RealPower','')
    return colname.replace('TFR','')
def mit_aggregate(sorted_array,name):
    sorted_array.drop_duplicates(subset = 'DATE_TIME', keep = 'first', inplace = True )
    times = sorted_array['DATE_TIME'].tolist()
    usage = sorted_array[name].tolist()
    aggregated_times = []
    aggregated_usage = []
    for i in range(len(times)-3):
        one = (times[i+1]-times[i] ==dt.timedelta(minutes=15))
        two = (times[i+2]-times[i+1] ==dt.timedelta(minutes=15))
        three = (times[i+3]-times[i+2] ==dt.timedelta(minutes=15))
        if(times[i].minute==0 and one and two and three):
            aggregated_times.append(times[i].strftime("%Y-%m-%d %H:%M:%S").encode('utf8'))
            aggregated_usage.append(str((usage[i]+usage[i+1]+usage[i+2]+usage[i+3])/4).encode('utf8'))
    return np.hstack((np.array(aggregated_times).reshape((-1,1)),np.array(aggregated_usage).reshape((-1,1))))
    
        

We can account by the error values by setting up a filter, and deal with the time issue by 1) sorting the dataframe before partitioning it and 2) writing the aggregate method

In [ ]:
ERROR_VALUES = ['No Data','I/O Timeout','Error','Pt Created','Configure']
with h5py.File(file_loc) as f:
    grp = f[home] if home in f else f.create_group(home)
    for dfile in subfiles: 
        year = dfile.split('\\')[-1].split('.')[0]
        df = pd.read_csv(dfile)
        print('Now analyzing data from: ................{}'.format(year))
        df.sort_values(by =['DATE_TIME'],inplace=True)
        dt_used = False
        df['DATE_TIME'] = pd.to_datetime(df['DATE_TIME'])
        for building in df.columns:
            if(not dt_used):
                dt_used = True
            else:
                building_name = mit_name_clean(building)
                print('Building.....{}'.format(building_name))
                subset = df[['DATE_TIME',building]]
                subset = subset[subset.apply(lambda x: str(x[building]) not in ERROR_VALUES,axis=1)]
                if len(subset) == 0:
                    continue
                else:
                    subset[building] = subset.apply(lambda x: float(x[building]), axis = 1)
                    cleaned = subset[subset[building]!=0]
                    dset_name ='{}/{}'.format(building_name,year)
                    grp[dset_name] = mit_aggregate(cleaned,building)
                
        

        

<h3><font color = #333f48>Metadata Transformation</font></h3>

<h2><font color =#bf5700>Ireland Dataset</font></h2>

In [17]:
home = 'ireland'                        #Insert home directory where data is stored
ext = '.txt'                            #Insert data storage type
separ = ' '                             #Separation
mdata = 'ireland/meta/metadata.csv'     #Point to location of metadata file

In [18]:
root, folders,x = [m for m in os.walk(home)][0]
data_files = [os.path.join(home,dfile) for dfile in x]
data_files

['ireland\\File1.txt',
 'ireland\\File2.txt',
 'ireland\\File3.txt',
 'ireland\\File4.txt',
 'ireland\\File5.txt',
 'ireland\\File6.txt']

The Ireland dataset is like the MIT dataset in that it too is shaped differently than our desires.
<ul>
<li>The files are segregated randomly, 1000 meters all years to a file instead of by building</li>
<li>The list is NOT ordered, meaning that time intervals next to each other in the original scrape will NOT necessarily be together in the final </li>
<li>In addition the separation between timestamps is not hourly as we would like but every 30 minutes </li>
<li> Most curiously, there is an interesting format this dataset uses for date time, of 'xxxyy' where 'xxx' is days after Dec. 31, 2008 and yy is 30 minute intervals. 00:00 - 00:30 is yy=01
</ul>

<h3><font color = #333f48>Data Transformation</font></h3>

In [19]:
#Ireland dataset stores date format in a weird form: 
def ireland_date(datum):
    #First three digits are the # of days after 31 Dec 2008
    dat = str(datum)
    num_days = int(float(dat[:3]))
    #Last three digits are the # of 30 minute intervals (1 = 0:0-0:30)
    times = int(float(dat[3:]))
    ORIGIN = dt.datetime(2008,12,31)
    DAY_INCREMENT = dt.timedelta(days=1)
    MIN_INCREMENT = dt.timedelta(minutes = 30)
    return ORIGIN + DAY_INCREMENT*num_days + MIN_INCREMENT*(times-1)
def aggregate(sorted_array):
    sorted_array.drop_duplicates(subset = 1, keep = 'first', inplace = True )
    times = sorted_array[1].dt.to_pydatetime().tolist()
    usage = sorted_array[2].tolist()
    aggregated_times = []
    aggregated_usage = []
    sets = {}
    current = 2009 
    changed = False
    for i in range(len(times)-1):
        if(times[i].year==2010 and not changed):
            changed = True
            sets[2009]=np.hstack((np.array(aggregated_times).reshape((-1,1)),np.array(aggregated_usage).reshape((-1,1))))
            aggregated_times = []
            aggregated_usage = []
        if(times[i].minute==0 and times[i+1]-times[i] ==dt.timedelta(minutes=30)):
            aggregated_times.append(times[i].strftime("%Y-%m-%d %H:%M:%S").encode('utf8'))
            aggregated_usage.append(str(usage[i]+usage[i+1]).encode('utf8'))
    sets[2010] = np.hstack((np.array(aggregated_times).reshape((-1,1)),np.array(aggregated_usage).reshape((-1,1))))
    return sets

Be very careful with running the script - it can take up to 2 hours!!!

In [ ]:
with h5py.File(file_loc) as f:
    counter = 0 
    for dfile in data_files:
        print('Now transcribing Data File: {}'.format(dfile))
        data = pd.read_csv(dfile,sep=separ,header=None)
        buildings = data[0].unique()
        for b in buildings:
            counter = counter + 1
            print('-----Building #{}:.........ID#:{}'.format(counter,b))
            subset = data[data[0]==b]
            subset[1] = subset.apply(lambda x: ireland_date(x[1]),axis = 1)
            subset.sort(1,inplace = True)
            sets = aggregate(subset)
            for key in sets.keys():
                f['ireland/{}/{}'.format(b,key)]=sets[key]
            

<h3><font color = #333f48>Metadata Transformation</font></h3>

In [21]:
def ireland_meta():
    TIMEZONE = "Europe/Cork"
    INDUSTRY = "Residential"
    metadata = pd.read_csv('IRELAND/meta/metadata.csv',encoding = 'ISO-8859-1',header=None)
    subset = metadata.loc[1:,[0,34,38,39]]
    with h5py.File(file_loc) as f:
        subset[0] = [str(x) for x in subset[0]]
        length = len(subset)
        i=0
        dset = f['ireland']
        for meterid in subset[0]:
            i = i+1
            print('Adding special metadata for Building #{}/{}............ID:{}'.format(i,length,meterid))
            grp = dset[meterid] if meterid in dset else None
            if grp == None:
                print('ID not found?')
            else:
                house_type = int(subset[subset[0]==meterid][34])
                if(house_type==1):
                    grp.attrs['PSU'] = 'Apartment'
                elif(house_type==2 or house_type==3 or house_type==4 or house_type==5):
                    grp.attrs['PSU'] = 'House'
                area = float(subset[subset[0]==meterid][38])
                if(area != 999999999):
                    sqft = area
                    units = int(subset[subset[0]==meterid][39])
                    if(units==1):
                        sqft = area*10.7639    #m^2 to sqft
                    grp.attrs['Sqft'] = sqft
        for mid in f['ireland'].keys():
            print(mid)
            grp = f['ireland/{}'.format(mid)] 
            grp.attrs['Timezone'] = TIMEZONE
            grp.attrs['Industry'] = INDUSTRY

    

In [ ]:
ireland_meta()

<h2><font color = #bf5700>Genome data set</font></h2>

In [ ]:
home = 'genome'                         #Insert home directory where data is stored
ext = '.csv'                            #Insert data storage type
separ = ' '                             #Separation
mdata = 'genome/meta_open.csv'          #Point to location of metadata file

<h2><font color = #bf5700>Pecan St. data set</font></h2>

In [23]:
import psycopg2 as ps

In [24]:
def create_connection(uid,pwd):
    HOST = '67.78.67.93'         #dataport.cloud
    PORT = 5434
    USER = uid   
    PWD = pwd    
    conn = None
    try:
        conn = ps.connect(dbname = "postgres", host = HOST, user = USER, password= PWD,port = PORT)
    except ps.Error as e:
        print ("connection error"+ e)
    return conn
def destroy_connection(conn):
    conn.close()

In [ ]:
def dfclean(df,clean_column):
    return df[df.apply(lambda x: x[clean_column] is not None,axis=1)]
def get_years(date_list):
    years = [date.year for date in date_list]
    return np.unique(years)
def get_building(data_id, conn):
    STMT = "SELECT localhour,use FROM university.electricity_egauge_hours WHERE dataid={}".format(data_id)
    try:
        cur = conn.cursor()
        cur.execute(STMT)
        rows = cur.fetchall()
        df = pd.DataFrame(rows)
        return df
    except:
        print('Querying Error')
    return pd.DataFrame()

In [ ]:
def get_ids():
    uniques_100=[22, 26, 48, 54, 59, 68, 77, 86, 93, 94, 101, 114, 115, 121, 130, 135, 160, 171, 187, 203, 222, 232, 243, 252,
    267, 275, 280, 297, 330, 347, 364, 370, 379, 410, 434, 436, 457, 470, 483, 484, 490, 491, 499, 503, 507, 508, 516, 527, 545,
    547, 555, 573, 575, 580, 585, 604, 621, 624, 645, 661, 668, 698, 739, 744, 765, 772, 774, 781, 796, 821, 861, 871, 878, 890,
    898, 900, 930, 936, 946, 954, 974, 980, 991, 994]
    uniques_200=[1037, 1069, 1086, 1103,1105,1153, 1167, 1169, 1185, 1192, 1202, 1283, 1310, 1314, 1331, 1334, 1350, 1354, 1392,
    1403, 1415, 1450, 1463, 1464, 1479, 1500, 1507, 1508, 1524, 1551, 1577, 1586, 1589, 1597, 1601, 1617, 1629, 1632, 1642,
    1681, 1696, 1697, 1700, 1714, 1718, 1731, 1766, 1782, 1790, 1791, 1792, 1796, 1800, 1801, 1830, 1832, 1845, 1854, 1879,
    1889, 1947, 1953, 1994]
    uniques_300 = [2004, 2018, 2031, 2034, 2062, 2072, 2075, 2094, 2129, 2144, 2156, 2158, 2171, 2199, 2204, 2207, 2233, 2242,
    2247, 2335, 2337, 2354, 2360, 2361, 2365, 2366, 2378, 2401, 2449, 2458, 2461, 2465, 2470, 2472, 2505, 2510, 2520, 2523, 2532,
    2557, 2575, 2606, 2638, 2641, 2667, 2710, 2742, 2750, 2751, 2755, 2769, 2787, 2814, 2815, 2818, 2824, 2829, 2845, 2859, 2864,
    2873, 2903, 2907, 2925, 2931, 2945, 2953, 2965, 2974, 2980, 2986, 2992, 2995]
    uniques_400 = [3009, 3032, 3036, 3039, 3044, 3087, 3092, 3104, 3126, 3134, 3143, 3160, 3192, 3204, 3215, 3221, 3224, 3235,
    3263, 3268, 3273, 3299, 3310, 3353, 3367, 3368, 3392, 3394, 3401, 3411, 3413, 3425, 3426, 3443, 3456, 3482, 3484, 3500,
    3504, 3506, 3510, 3519, 3527, 3531, 3538, 3544, 3577, 3615, 3631, 3632, 3635, 3649, 3652, 3676, 3678, 3687, 3719, 3721,
    3723, 3734, 3736, 3778, 3789, 3795, 3806, 3829, 3831, 3849, 3864, 3873, 3883, 3886, 3893, 3916, 3918, 3935, 3938, 3953,
    3964, 3967, 3973]
    uniques_500 = [4000, 4022, 4031, 4042, 4053, 4083, 4095, 4135, 4147, 4154, 4193, 4213, 4220, 4224, 4251, 4296, 4297, 4298,
    4302, 4313, 4321, 4329, 4336, 4342, 4352, 4357, 4373, 4375, 4383, 4416, 4438, 4447, 4473, 4495, 4499, 4505, 4514, 4526,
    4544, 4575, 4590, 4601, 4633, 4641, 4660, 4670, 4674, 4699, 4703, 4732, 4761, 4767, 4773, 4776, 4800, 4830, 4856, 4864,
    4874, 4910, 4920, 4922, 4927, 4934, 4944, 4946, 4956, 4957, 4967, 4974, 4998]
    uniques_600 = [5009, 5026, 5035, 5060, 5087, 5109, 5129, 5164, 5187, 5209, 5218, 5226, 5246, 5252, 5262, 5271, 5275, 5279,
    5288, 5298, 5317, 5356, 5357, 5371, 5395, 5400, 5403, 5438, 5439, 5448, 5449, 5450, 5456, 5485, 5539, 5545, 5552, 5568,
    5615, 5652, 5658, 5673, 5677, 5718, 5728, 5738, 5746, 5749, 5759, 5778, 5784, 5785, 5786, 5796, 5809, 5810, 5814, 5817,
    5852, 5874, 5889, 5892, 5904, 5909, 5921, 5938, 5944, 5949, 5959, 5972, 5994]
    uniques_700=[6012, 6061, 6063, 6072, 6078, 6083, 6101, 6108, 6121, 6125, 6139, 6148, 6165, 6174, 6191, 6248, 6264, 6266, 6268,
    6286, 6324, 6334, 6348, 6377, 6378, 6412, 6418, 6423, 6429, 6460, 6497, 6498, 6500, 6536, 6545, 6547, 6578, 6593, 6614, 6636,
    6643, 6673, 6688, 6689, 6691, 6692, 6730, 6799, 6800, 6826, 6836, 6871, 6887, 6888, 6910, 6911, 6941, 6956, 6960, 6979, 6990]
    uniques_800 = [7001, 7013, 7016, 7017, 7024, 7030, 7036, 7057, 7062, 7108, 7114, 7117, 7122, 7166, 7208, 7240, 7276, 7287, 7319,
    7361, 7390, 7408, 7409, 7429, 7436, 7468, 7491, 7504, 7510, 7512, 7527, 7531, 7536, 7541, 7549, 7560, 7585, 7587, 7597, 7617,
    7627, 7638, 7639, 7641, 7680, 7693, 7703, 7719, 7731, 7739, 7741, 7764, 7767, 7769, 7787, 7788, 7792, 7793, 7794, 7800, 7818,
    7850, 7863, 7866, 7875, 7881, 7893, 7900, 7901, 7940, 7951, 7965, 7973, 7982, 7984, 7989]
    uniques_900 = [8029, 8031, 8034, 8046, 8047, 8059, 8061, 8071, 8079, 8084, 8086, 8092, 8117, 8121, 8122, 8142, 8155, 8156, 8163,
    8183, 8188, 8197, 8198, 8201, 8218, 8236, 8243, 8273, 8282, 8292, 8317, 8328, 8342, 8368, 8386, 8395, 8419, 8467, 8555, 8565,
    8574, 8589, 8597, 8600, 8622, 8626, 8645, 8669, 8729, 8730, 8733, 8736, 8741, 8767, 8807, 8829, 8847, 8848, 8852, 8857, 8862,
    8872, 8886, 8890, 8942, 8956, 8961, 8967, 8986, 8995]
    uniques_1000=[9001, 9019, 9036, 9052, 9085, 9121, 9134, 9139, 9141, 9142, 9156, 9160, 9165, 9182, 9195, 9201, 9206, 9213, 9215,
    9233, 9235, 9237, 9248, 9251, 9277, 9278, 9295, 9333, 9340, 9341, 9343, 9356, 9370, 9434, 9451, 9462, 9484, 9488, 9498, 9499,
    9509, 9548, 9555, 9578, 9585, 9605, 9609, 9610, 9612, 9613, 9624, 9631, 9642, 9643, 9647, 9654, 9670, 9674, 9688, 9701, 9729,
    9737, 9745, 9766, 9771, 9773, 9775, 9776, 9803, 9818, 9830, 9836, 9846, 9875, 9912, 9915, 9919, 9921, 9922, 9923, 9926, 9929,
    9931, 9932, 9933, 9934, 9935, 9936, 9937, 9938, 9939, 9942, 9958, 9971, 9981, 9982, 9983]
    return uniques_100 + uniques_200 + uniques_300 + uniques_400 + uniques_500 + uniques_600 + uniques_700 + uniques_800 + uniques_900 + uniques_1000

Be very careful! Code can run as fast as 30 mintues to slightly over 2 hours depends on internet connection

In [ ]:
user = str(input('Please enter Username:'))
pwd = str(input('Please enter Password:'))
conn = create_connection(user,pwd)

with h5py.File(file_loc) as f:
    pecan = f['pecan'] if 'pecan' in f else f.create_group('pecan')
    i = 0 
    for building_id in get_ids():
        i = i+1 
        print('Building #{}/747 : ID #{}.....'.format(i,building_id))
        df = get_building(building_id,conn)
        if(df.empty):
                print('Empty dataset')
        else:
            grp = None
            if '{}'.format(building_id) not in pecan:
                grp = pecan.create_group(str(building_id))
            else:
                grp = pecan['{}'.format(building_id)] 
            years = get_years(df[0].tolist())
            for year in years:
                print('...Adding Dataset - {}'.format(year))
                subset = dfclean(df[df.apply(lambda x: x[0].year==year,axis=1)],1)
                date_list = [date.strftime("%Y-%m-%d %H:%M:%S").encode('utf8') for date in subset[0].tolist()]
                usage = [float(x) for x in subset[1].tolist()]
                dataset = np.hstack((np.array(date_list,dtype=np.string_).reshape((-1,1)),np.array(usage,dtype=np.float64).reshape((-1,1))))
                grp[str(year)]=dataset


In [25]:
def simplifying_function_2011(x):
    if(x[2]==None or x[3]==None or x[4]==None or x[5] ==None or x[6]==None or x[7]==None):
        return None
    elif(x[2]==0 and x[3]==0 and x[4]==0 and x[5]==0 and x[6]==0 and x[7]==0):
        return 'Apartment'
    else:
        return 'House'
def add_mdata():
    TIMEZONE = "America/Austin"
    INDUSTRY = "Residential"
    STMT_2013 = 'SELECT dataid,"Conditioned_Square_Footage__c","Type_of_Home__c" FROM university.audits_2013_main'
    STMT_2011 = 'SELECT dataid,conditions_square_foot,type_of_home_single_family,type_of_home_duplex,type_of_home_triplex,type_of_home_four_plex,type_of_home_condo,type_of_home_town_home FROM university.audits_2011'
    df_2013 = pd.DataFrame()
    df_2011 = pd.DataFrame()
    user = str(input('Please enter Username:'))
    pwd = str(input('Please enter Password:'))
    try:
        conn = create_connection(user,pwd)
        cur = conn.cursor()
        cur.execute(STMT_2011)
        rows = cur.fetchall()
        destroy_connection(conn)
        df_2011 = pd.DataFrame(rows)
    except ps.Error:
        print('Querying Error_2011')
    try:
        conn = create_connection(user,pwd)
        cur = conn.cursor()
        cur.execute(STMT_2013)
        rows = cur.fetchall()
        destroy_connection(conn)
        df_2013 = pd.DataFrame(rows)
    except ps.Rrror:
        print('Querying Error_2013')
    house = df_2011.apply(simplifying_function_2011,axis=1)
    df_2011[2] = house
    subset_2011 = df_2011.loc[:,0:2]
    df_2013[2]=[x if x=='Apartment' else 'House'for x in df_2013[2].tolist()]
    union = df_2011.merge(df_2013,how='outer',left_on=0,right_on=0)
    lists = get_ids()
    with h5py.File(file_loc) as f:
        for index,row in union.iterrows():
            bid = int(row[0])
            print(bid)
            if bid in lists:
                grp = f['pecan/{}'.format(bid)]
                if(pd.isnull(row['2_y']) and not pd.isnull(row['2_x'])):
                    grp.attrs['PSU'] = row['2_x']
                elif(not pd.isnull(row['2_y'])):
                    grp.attrs['PSU']=row['2_y']
                if(pd.isnull(row['1_y']) and not pd.isnull(row['1_x'])):
                    grp.attrs['Sqft'] = float(row['1_x'])
                elif(not pd.isnull(row['1_y'])):
                    grp.attrs['Sqft']=float(row['1_y'])
        for key in f.keys():
            grp =f[key]
            grp.attrs['Timezone'] = TIMEZONE
            grp.attrs['Industry'] = INDUSTRY
        

In [ ]:
add_mdata()